## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
def cal_undistort(img, objpoints, imgpoints):
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints,\
                                                      imgpoints, img.shape[1:], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

In [2]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
#%matplotlib qt
%matplotlib inline

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')
#images = glob.glob('../test_images/test1.jpg')


# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    # Convert to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    nx = 9
    ny = 6

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Undistort using mtx and dist
        undist = cal_undistort(img, objpoints, imgpoints)
        
        
        offset = 100 # offset for dst points
        img_size = (gray.shape[1], gray.shape[0])
        
        src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])
        dst = np.float32([[offset, offset], [img_size[0]-offset, offset],\
                         [img_size[0]-offset, img_size[1]-offset],\
                         [offset, img_size[1]-offset]])
        M = cv2.getPerspectiveTransform(src, dst)
        
        warped = cv2.warpPerspective(undist, M, img_size)
        
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
        #cv2.imshow('img',img)
        #plt.imshow(img)
        #cv2.waitKey(500) 
        
#cv2.destroyAllWindows()
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24,9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(warped)
ax2.set_title('Undistorted Imgae', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

ModuleNotFoundError: No module named 'cv2'

# Pipeline

## 2. Describe how (and identify where in your code) you used color transforms, gradients or other methods to create a thresholded binary image. Provide an example of a binary image result.

In [3]:
def mag_select(img, sobel_kernel=3, mag_thresh=(0,255)):
    # 1) Convert to grayscale
    gray_test_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray_test_img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray_test_img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # 3) Calculate the magnitude
    abs_sobelxy = np.sqrt(np.square(sobelx)+np.square(sobely))
    
    # 4) Scale to 8-bit (0-255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobelxy/np.max(abs_sobelxy))
    
    # 5) Create a binary mask where mag thresholds are met
    thresh_min = mag_thresh[0]
    thresh_max = mag_thresh[1]
    
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    
    return binary_output

def s_select(img, s_thresh=(0,255)):
    #1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    #2) Apply a threshold to the S channel
    S = hls[:,:,2]
    
    #3) Return a binary image of threshold result
    binary_output = np.zeros_like(S)
    binary_output[(S > s_thresh[0]) & (S <= s_thresh[1])] = 1
    
    return binary_output

def h_select(img, h_thresh=(0,255)):
    #1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    #2) Apply a threshold to the H channel
    H = hls[:,:,0]
    
    #3) Return a binary image of threshold result
    binary_output = np.zeros_like(H)
    binary_output[(H > h_thresh[0]) & (H <= h_thresh[1])] = 1
    
    return binary_output

def threshold_pipeline(img, h_thresh=(18,100), s_thresh=(90,255),\
                      mag_thresh=(20,100)):
    # Magnitude 
    mag_binary = mag_select(img, sobel_kernel=3, mag_thresh=(20,100))
    print('mag size:{}'.format(mag_binary.shape))
    
    # Saturation
    s_binary = s_select(img, s_thresh=(80,255))
    print('s size:{}'.format(s_binary.shape))
        
    # Hue
    h_binary = h_select(img, h_thresh=(20,100))
    print('h size:{}'.format(h_binary.shape))
        
    # Stack each channel
    result = np.dstack((np.zeros_like(mag_binary), mag_binary, s_binary, h_binary))*255
    
    return result

In [4]:
from PIL import Image

test_img = np.array(Image.open('../test_images/test2.jpg'))

s_binary = s_select(test_img, s_thresh=(80, 255))

h_binary = h_select(test_img, h_thresh=(18,100))

mag_binary = mag_select(test_img, sobel_kernel=3, mag_thresh=(20, 100))

combined = np.zeros_like(test_img)
combined = threshold_pipeline(test_img)

print('combined size:{}'.format(combined.shape))

f1, [(ax11, ax12), (ax21, ax22), (ax31, ax32)] = plt.subplots(3, 2, figsize=(24, 9))
f1.tight_layout()

ax11.imshow(test_img)
ax11.set_title('Original Test Image', fontsize=30)

ax12.imshow(mag_binary, cmap='gray')
ax12.set_title('Thresholded Mag', fontsize=30)

ax21.imshow(s_binary, cmap='gray')
ax21.set_title('Thresholded S', fontsize=30)

ax22.imshow(h_binary, cmap='gray')
ax22.set_title('Thresholded H', fontsize=30)

ax31.imshow(combined, cmap='gray')
ax31.set_title('Thresholded Combined', fontsize=30)

plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0., \
                    wspace=0., hspace=0.5)

NameError: name 'cv2' is not defined